In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [2]:
df = pd.read_csv("raw_ksi_data.csv")

In [3]:
df.shape

(18957, 54)

In [4]:
df.sample(2)

,OBJECTID,INDEX,ACCNUM,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,x,y
8325,8326,7441959,1305685.0,6/6/2012 8:00:00 AM,2231,YONGE ST,CARLTON ST,NaN,Major Arterial,Toronto and East York,...,NaN,NaN,NaN,170,Yonge-Bay Corridor,76,Bay Street Corridor (76),D52,630387.9419,4.835463e+06
3881,3882,5417329,1063638.0,9/24/2008 8:00:00 AM,2035,319 KIPLING AVE,NaN,NaN,Major Arterial,Etobicoke York,...,NaN,NaN,Yes,18,New Toronto,18,New Toronto (18),D22,619665.2586,4.828419e+06


In [5]:
df.drop(['ACCNUM','OBJECTID', 'HOOD_140', 'NEIGHBOURHOOD_140', 'OFFSET', 'ROAD_CLASS', 'ACCLOC', 'INVAGE', 'INITDIR'], axis=1, inplace=True)

In [6]:
df.drop(['VEHTYPE', 'MANOEUVER', 'DRIVACT', 'DRIVCOND', 'PEDTYPE', 'PEDACT', 'CYCLISTYPE', 'CYCACT', 'CYCCOND', 'PEDESTRIAN', "TRSN_CITY_VEH"], axis=1, inplace=True)

In [7]:
df.drop(['EMERG_VEH', 'PASSENGER', 'REDLIGHT', 'DISABILITY', 'DIVISION', 'SPEEDING', 'PEDCOND', 'AG_DRIV','x', 'y' ], axis=1, inplace=True)

In [8]:
df.drop(['CYCLIST', 'IMPACTYPE', 'INVTYPE', 'INDEX', 'TRAFFCTL', 'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK', 'ALCOHOL'], axis=1, inplace=True)

In [9]:
df.shape

(18957, 15)

In [10]:
df.sample(2)

,DATE,TIME,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,INJURY,FATAL_NO,HOOD_158,NEIGHBOURHOOD_158
6672,1/11/2011 10:00:00 AM,1140,DUNDAS ST W,BAY ST,Toronto and East York,43.655745,-79.38379,Clear,Daylight,Dry,Non-Fatal Injury,NaN,NaN,170,Yonge-Bay Corridor
5029,9/20/2009 8:00:00 AM,244,DUNDAS ST W,COOLMINE RD,Toronto and East York,43.649445,-79.42559,Clear,Dark,Dry,Non-Fatal Injury,Major,NaN,84,Little Portugal


In [11]:
df['TIME'] = df['TIME'].astype(str).str.zfill(4)
df['DATE'] = pd.to_datetime(df['DATE'], format='%m/%d/%Y %I:%M:%S %p')

# Extract DOW and HOUR and YEAR
df['DOW'] = df['DATE'].dt.day_name() 
df['HOUR'] = df['TIME'].str[:2].astype(int)
df['YEAR'] = df['DATE'].dt.year
df['MONTH'] = df['DATE'].dt.month

In [12]:
df.sample(3)

,DATE,TIME,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,INJURY,FATAL_NO,HOOD_158,NEIGHBOURHOOD_158,DOW,HOUR,YEAR,MONTH
5086,2009-10-02 08:00:00,1626,FINCH AVE W,KEELE ST,Etobicoke York,43.763445,-79.490990,Rain,Daylight,Wet,Non-Fatal Injury,NaN,NaN,27,York University Heights,Friday,16,2009,10
2545,2007-09-14 08:00:00,0820,AUKLAND RD,SUBWAY CRES W,Etobicoke York,43.638345,-79.537690,Clear,Daylight,Dry,Non-Fatal Injury,NaN,NaN,159,Etobicoke City Centre,Friday,8,2007,9
16973,2021-03-18 08:00:00,1056,GREENWIN VILLAGE RD,BATHURST ST,North York,43.790755,-79.444835,Clear,Daylight,Dry,Non-Fatal Injury,Major,NaN,36,Newtonbrook West,Thursday,10,2021,3


In [13]:
df["INJURY"].unique()

array(['Major', 'Minor', nan, 'Fatal', 'Minimal'], dtype=object)

In [14]:
df["INJURY"].value_counts()

INJURY
Major      6445
Minor      1479
Minimal    1160
Fatal       976
Name: count, dtype: int64

In [15]:
df['INJURY'] = df['INJURY'].fillna('Minimal')

In [16]:
df["ACCLASS"].unique()

array(['Non-Fatal Injury', 'Fatal', 'Property Damage O', nan],
      dtype=object)

In [17]:
df["ACCLASS"].value_counts()

ACCLASS
Non-Fatal Injury     16268
Fatal                 2670
Property Damage O       18
Name: count, dtype: int64

In [18]:
df['ACCLASS'] = df['ACCLASS'].replace('Property Damage O', 'Non-Fatal Injury')
df['ACCLASS'] = df['ACCLASS'].fillna('Non-Fatal Injury')

In [19]:
df['SEVERE_COLLISION'] = (
    (df['INJURY'].isin(['Major', 'Fatal'])) | 
    (df['ACCLASS'] == 'Fatal')
).astype(int)

In [20]:
df['FATAL_NO'] = df['FATAL_NO'].fillna(0)

In [21]:
df["VISIBILITY"].value_counts()

VISIBILITY
Clear                     16373
Rain                       1976
Snow                        356
Other                        98
Fog, Mist, Smoke, Dust       52
Freezing Rain                47
Drifting Snow                21
Strong wind                  10
Name: count, dtype: int64

In [22]:
df['VISIBILITY'] = df['VISIBILITY'].replace({
    'Other': 'Clear',
    'Freezing Rain': 'Rain',
    'Strong wind': 'Rain',
    'Fog, Mist, Smoke, Dust': 'Rain',
    'Drifting Snow': 'Snow'
})

In [23]:
df["VISIBILITY"].value_counts()

VISIBILITY
Clear    16471
Rain      2085
Snow       377
Name: count, dtype: int64

In [24]:
df["LIGHT"].value_counts()

LIGHT
Daylight                10779
Dark                     3746
Dark, artificial         3552
Dusk                      253
Dusk, artificial          253
Daylight, artificial      151
Dawn                      112
Dawn, artificial          101
Other                       6
Name: count, dtype: int64

In [25]:
df['LIGHT'] = df['LIGHT'].replace({
    'Daylight, artificial': 'Daylight',
    'Dark, artificial': 'Dark',
    'Dusk, artificial': 'Dusk',
    'Dawn, artificial': 'Dawn',
    'Other': 'Daylight'
})

In [26]:
df["LIGHT"].value_counts()

LIGHT
Daylight    10936
Dark         7298
Dusk          506
Dawn          213
Name: count, dtype: int64

In [27]:
df.rename(columns={'RDSFCOND': 'ROAD_CONDITION'}, inplace=True)

In [28]:
df["ROAD_CONDITION"].value_counts()

ROAD_CONDITION
Dry                     15231
Wet                      3140
Loose Snow                174
Other                     147
Slush                     102
Ice                        78
Packed Snow                44
Loose Sand or Gravel       11
Spilled liquid              1
Name: count, dtype: int64

In [29]:
df['ROAD_CONDITION'] = df['ROAD_CONDITION'].replace({
    'Spilled liquid': 'Wet',
    'Loose Snow': 'Snow_Ice',
    'Packed Snow': 'Snow_Ice',
    'Slush': 'Snow_Ice',
    'Ice': 'Snow_Ice',
    'Loose Sand or Gravel': 'Dry',
    'Other': 'Dry'
})

In [30]:
df["ROAD_CONDITION"].value_counts()

ROAD_CONDITION
Dry         15389
Wet          3141
Snow_Ice      398
Name: count, dtype: int64

In [31]:
df['DATE'] = pd.to_datetime(df['DATE']).dt.date

In [32]:
df.drop(['TIME'], axis=1, inplace=True)#since hour col, exact time not needed

In [33]:
df.sample(5)

,DATE,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,ROAD_CONDITION,ACCLASS,INJURY,FATAL_NO,HOOD_158,NEIGHBOURHOOD_158,DOW,HOUR,YEAR,MONTH,SEVERE_COLLISION
15780,2019-07-29,GERRARD ST E,CARLAW AVE,Toronto and East York,43.667624,-79.342769,Clear,Daylight,Dry,Non-Fatal Injury,Minimal,0.0,70,South Riverdale,Monday,19,2019,7,0
5164,2009-10-20,ALBION RD,ARMEL CRT,Etobicoke York,43.721345,-79.545590,Clear,Daylight,Dry,Non-Fatal Injury,Minimal,0.0,5,Elms-Old Rexdale,Tuesday,12,2009,10,0
14953,2018-09-16,LAWSON RD,ST MAGNUS DR,Scarborough,43.787551,-79.150020,Clear,Dark,Dry,Non-Fatal Injury,Major,0.0,133,Centennial Scarborough,Sunday,4,2018,9,1
3095,2008-02-17,1601 LAKE SHORE BLVD W,NaN,Toronto and East York,43.636945,-79.446890,Clear,Dark,Dry,Non-Fatal Injury,Minor,0.0,85,South Parkdale,Sunday,2,2008,2,0
14212,2017-12-29,ST CLAIR AVE E,KENNEDY RD,Scarborough,43.716452,-79.261302,Snow,Daylight,Snow_Ice,Non-Fatal Injury,Major,0.0,120,Clairlea-Birchmount,Friday,15,2017,12,1


In [34]:
def categorize_hour(hour):
    if 0 <= hour < 4:
        return 'Late Night'
    elif 4 <= hour < 8:
        return 'Early Morning'
    elif 8 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 16:
        return 'Afternoon'
    elif 16 <= hour < 20:
        return 'Evening Rush'
    else:
        return 'Night'

In [35]:
def categorize_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

In [36]:
df['TIME_OF_DAY'] = df['HOUR'].apply(categorize_hour)
df['SEASON'] = df['MONTH'].apply(categorize_season)

In [37]:
df['ROAD_CONDITION'] = df['ROAD_CONDITION'].replace(['', ' ', 'nan', 'NaN'], np.nan)
df.loc[df['ROAD_CONDITION'].isna() & (df['SEVERE_COLLISION'] == 1), 'ROAD_CONDITION'] = 'Wet'
df.loc[df['ROAD_CONDITION'].isna() & (df['SEVERE_COLLISION'] == 0), 'ROAD_CONDITION'] = 'Dry'

In [38]:
df['LIGHT'] = df['LIGHT'].replace(['', ' ', 'nan', 'NaN'], np.nan)
df.loc[df['LIGHT'].isna() & (df['SEVERE_COLLISION'] == 1), 'LIGHT'] = 'Dark'
df.loc[df['LIGHT'].isna() & (df['SEVERE_COLLISION'] == 0), 'LIGHT'] = 'Daylight'

In [39]:
df['VISIBILITY'] = df['VISIBILITY'].replace(['', ' ', 'nan', 'NaN'], np.nan)
df.loc[df['VISIBILITY'].isna() & (df['SEVERE_COLLISION'] == 1), 'VISIBILITY'] = 'Rain'
df.loc[df['VISIBILITY'].isna() & (df['SEVERE_COLLISION'] == 0), 'VISIBILITY'] = 'Clear'

In [40]:
df.shape

(18957, 21)

In [41]:
df.to_csv('cleaned_ksi_collisions.csv', index=False)